In [ ]:
# ============================================================================
# DETECCIÓN DEL ENTORNO - Esta sección debe ejecutarse primero
# ============================================================================
try:
    import google.colab
    IN_COLAB = True
    print("✓ Ejecutando en Google Colab")
except:
    IN_COLAB = False
    print("✓ Ejecutando en entorno local")

# ============================================================================
# CARGA DE ARCHIVO DE MUNICIPIOS
# ============================================================================
import geopandas as gpd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

if IN_COLAB:
    from google.colab import files
    print("\n📁 Por favor suba su archivo de municipios:")
    print("   Formatos aceptados: .gpkg, .shp (+ archivos auxiliares), .geojson")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    print(f"\n✓ Archivo subido: {filename}")
else:
    # Si no estás en Colab, especifica la ruta de tu archivo
    filename = 'data/municipios_seleccionados.gpkg'
    print(f"✓ Usando archivo local: {filename}")

# Cargar con geopandas
print("\nCargando archivo...")
municipios_gdf = gpd.read_file(filename)
print(f"✓ Archivo cargado exitosamente")
print(f"  Municipios: {len(municipios_gdf)}")
print(f"  CRS: {municipios_gdf.crs}")
print(f"  Columnas: {', '.join(municipios_gdf.columns.drop('geometry'))}")

# Reproyectar a WGS84 si es necesario
if municipios_gdf.crs != 'EPSG:4326':
    print(f"\n  Reproyectando a WGS84...")
    municipios_gdf = municipios_gdf.to_crs('EPSG:4326')
    print(f"  ✓ Reproyectado a EPSG:4326")

# Mostrar primeros registros
print(f"\n📊 Primeros municipios:")
print(municipios_gdf.head())

In [1]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generación de Mosaicos Sentinel-2 Libres de Nubes por Municipio

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/famartinezal/sentinel1-change-detection/blob/master/05_mosaicos_sentinel2_sin_nubes.ipynb)

**Autor**: Adaptado del tutorial s2cloudless de jdbcode  

Este notebook permite generar mosaicos ópticos libres de nubes utilizando Sentinel-2 para cualquier conjunto de municipios que suban en formato GeoPackage, Shapefile o GeoJSON.

## Introducción

Este notebook implementa la generación de mosaicos ópticos libres de nubes utilizando imágenes Sentinel-2 para municipios en Colombia. El enmascaramiento de nubes se realiza mediante el dataset **s2cloudless**, que proporciona probabilidades de nubosidad pixel a pixel, y la detección de sombras mediante proyección de nubes e intersección con píxeles oscuros en la banda NIR.

### Características

- **Diseñado para Google Colab**: No requiere instalación local
- **Sube tu propio archivo**: Acepta GeoPackage (.gpkg), Shapefile (.shp) o GeoJSON (.geojson)
- **Parámetros configurables**: Ajusta fechas y umbrales de enmascaramiento
- **Índices espectrales**: Calcula automáticamente NDVI, NDWI y EVI
- **Visualización interactiva**: Mapas con folium para explorar resultados
- **Descarga de estadísticas**: CSV con resumen de imágenes por municipio

### Flujo de trabajo

1. Autenticar Google Earth Engine
2. Instalar dependencias (geemap)
3. Subir archivo de municipios
4. Configurar fechas y parámetros
5. Procesar colección Sentinel-2
6. Generar mosaicos por municipio
7. Visualizar y descargar resultados

### Run me first

Run the following cell to initialize the Earth Engine API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [ ]:
# Detectar entorno (Colab o local)
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# Instalar geemap si estamos en Colab
if IN_COLAB:
    print("Ejecutando en Google Colab - Instalando dependencias...")
    !pip install -q geemap
    print("✓ geemap instalado")

import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-famageoia')  # Reemplaza con tu project ID

print("✓ Earth Engine inicializado")

## 1. Cargar Archivo de Municipios

Sube un archivo con los límites de tus municipios en formato **GeoPackage** (.gpkg), **Shapefile** (.shp + archivos auxiliares), o **GeoJSON** (.geojson).

**Requisitos:**
- Geometrías tipo polígono
- Sistema de coordenadas: cualquiera (se reproyecta automáticamente a WGS84)
- Campos sugeridos (nombres flexibles): `NOMBRE`, `CODIGO`, `DEPARTAMENTO`

In [ ]:
# ============================================================================
# CONVERTIR POLÍGONO(S) CARGADO(S) A AOI DE EARTH ENGINE
# ============================================================================

print("="*70)
print("ÁREA DE INTERÉS (AOI)")
print("="*70)

# El archivo cargado es el AOI completo
# Si tiene múltiples polígonos, se unifican en un solo AOI

print(f"  Número de polígonos: {len(municipios_gdf)}")

# Calcular área total y centroide
total_area_ha = municipios_gdf.geometry.area.sum() / 10000  # m² a hectáreas
aoi_unified = municipios_gdf.unary_union  # Unificar todos los polígonos
centroid = aoi_unified.centroid

print(f"  Área total: {total_area_ha:.2f} hectáreas ({total_area_ha/100:.2f} km²)")
print(f"  Centroide: ({centroid.y:.4f}, {centroid.x:.4f})")

# Convertir toda la geometría a Earth Engine
import json
geom_geojson = json.loads(municipios_gdf.to_json())

# Crear ee.Geometry desde todas las features
all_features = []
for feature in geom_geojson['features']:
    geom_coords = feature['geometry']
    if geom_coords['type'] == 'Polygon':
        all_features.append(ee.Geometry.Polygon(geom_coords['coordinates']))
    elif geom_coords['type'] == 'MultiPolygon':
        all_features.append(ee.Geometry.MultiPolygon(geom_coords['coordinates']))

# Si hay múltiples geometrías, unirlas
if len(all_features) == 1:
    AOI = all_features[0]
else:
    # Unir todos los polígonos en un solo MultiPolygon
    AOI = ee.Geometry.MultiPolygon([geom.coordinates().getInfo() for geom in all_features])

print(f"  Tipo de geometría EE: {AOI.type().getInfo()}")

# Nombre para el área (usar nombre de columna si existe, sino genérico)
if len(municipios_gdf) == 1:
    area_nombre = municipios_gdf.iloc[0].get('MPIO_CNMBR', 
                  municipios_gdf.iloc[0].get('NOMBRE',
                  municipios_gdf.iloc[0].get('nombre', 'Area_de_interes')))
else:
    area_nombre = f'AOI_{len(municipios_gdf)}_poligonos'

print(f"\n✓ AOI configurado: {area_nombre}")
print("="*70)

## 2. Configurar Área de Interés (AOI)

El archivo que subiste define el **área de interés (AOI)** completa. Todos los polígonos en el archivo se procesan como una sola área.


In [ ]:
# ============================================================================
# CONFIGURACIÓN DE PARÁMETROS TEMPORALES Y DE ENMASCARAMIENTO
# ============================================================================

# PARÁMETROS TEMPORALES
# Modifica estas fechas según tu periodo de interés
START_DATE = '2024-07-01'  # Fecha inicio (inclusive)
END_DATE = '2024-12-31'    # Fecha fin (exclusiva)

# PARÁMETROS DE FILTRADO Y ENMASCARAMIENTO
CLOUD_FILTER = 70          # % máximo cobertura de nubes en la imagen
CLD_PRB_THRESH = 40        # Umbral probabilidad de nube (más bajo = más agresivo)
NIR_DRK_THRESH = 0.15      # Umbral reflectancia NIR para sombras
CLD_PRJ_DIST = 2           # Distancia proyección de sombras (km)
BUFFER = 100               # Buffer para dilatar nubes detectadas (m)

print("="*70)
print("PARÁMETROS CONFIGURADOS")
print("="*70)
print(f"  Área:                {area_nombre}")
print(f"  Fecha inicio:        {START_DATE}")
print(f"  Fecha fin:           {END_DATE}")
print(f"  Filtro nubes:        {CLOUD_FILTER}%")
print(f"  Umbral prob. nube:   {CLD_PRB_THRESH}%")
print(f"  Umbral NIR oscuro:   {NIR_DRK_THRESH}")
print(f"  Distancia proyección: {CLD_PRJ_DIST} km")
print(f"  Buffer:              {BUFFER} m")
print("="*70)


## 4. Construir Colección Sentinel-2 Filtrada

Se construye la colección Sentinel-2 filtrando por:
1. Límites del municipio seleccionado
2. Rango de fechas especificado
3. Cobertura de nubes máxima permitida

La función `get_s2_sr_cld_col()` combina dos colecciones:
- **COPERNICUS/S2_SR**: Reflectancia superficial Sentinel-2
- **COPERNICUS/S2_CLOUD_PROBABILITY**: Probabilidades de nubosidad pixel a pixel (s2cloudless)


In [ ]:
# ============================================================================
# FUNCIÓN PARA CONSTRUIR COLECCIÓN SENTINEL-2 CON S2CLOUDLESS
# ============================================================================

def get_s2_sr_cld_col(aoi, start_date, end_date, cloud_filter):
    """
    Construye una colección Sentinel-2 SR con datos de probabilidad de nubes.
    
    Args:
        aoi: ee.Geometry - Área de interés
        start_date: str - Fecha inicio 'YYYY-MM-DD'
        end_date: str - Fecha fin 'YYYY-MM-DD'
        cloud_filter: int - % máximo de cobertura de nubes
    
    Returns:
        ee.ImageCollection - Colección SR con propiedad 's2cloudless'
    """
    # Importar y filtrar Sentinel-2 SR
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))

    # Importar y filtrar s2cloudless
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Unir ambas colecciones por 'system:index'
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

print("✓ Función get_s2_sr_cld_col() definida")


In [ ]:
# Construir la colección filtrada por el municipio y fechas
print("\n🔍 Construyendo colección Sentinel-2...")
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE, CLOUD_FILTER)

# Obtener información de la colección
n_images = s2_sr_cld_col.size().getInfo()
print(f"✓ Colección construida: {n_images} imágenes encontradas")

if n_images == 0:
    print("\n⚠️  ADVERTENCIA: No se encontraron imágenes con los criterios especificados.")
    print("    Intente:")
    print("    - Ampliar el rango de fechas")
    print("    - Aumentar CLOUD_FILTER (permitir más nubes)")
    print("    - Verificar que el municipio tenga cobertura Sentinel-2")
else:
    # Mostrar fechas de imágenes disponibles
    dates_list = s2_sr_cld_col.aggregate_array('system:time_start').getInfo()
    import datetime
    dates = [datetime.datetime.fromtimestamp(d/1000).strftime('%Y-%m-%d') for d in dates_list[:10]]
    print(f"\n📅 Primeras fechas disponibles: {', '.join(dates)}")
    if n_images > 10:
        print(f"    ... y {n_images - 10} más")


In [ ]:
# ============================================================================
# FUNCIONES DE ENMASCARAMIENTO
# ============================================================================

def add_cloud_bands(img):
    """Agrega bandas de probabilidad de nube y máscara binaria de nubes."""
    # Obtener probabilidad de nube desde s2cloudless
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    
    # Generar máscara binaria basada en umbral
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    
    # Agregar bandas a la imagen
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    """Agrega bandas de detección de sombras de nubes."""
    # Identificar píxeles de agua (para excluirlos de sombras)
    not_water = img.select('SCL').neq(6)
    
    # Identificar píxeles oscuros en NIR (sombras potenciales)
    SR_BAND_SCALE = 1e4
    dark_pixels = (img.select('B8')
                   .lt(NIR_DRK_THRESH * SR_BAND_SCALE)
                   .multiply(not_water)
                   .rename('dark_pixels'))
    
    # Calcular azimut de proyección de sombras
    shadow_azimuth = ee.Number(90).subtract(
        ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    )
    
    # Proyectar sombras desde nubes
    cld_proj = (img.select('clouds')
                .directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST * 10)
                .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                .select('distance')
                .mask()
                .rename('cloud_transform'))
    
    # Intersección de píxeles oscuros con proyección de sombras
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    
    # Agregar bandas de sombras
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    """Genera la máscara final combinando nubes y sombras."""
    # Agregar componentes de nubes
    img_cloud = add_cloud_bands(img)
    
    # Agregar componentes de sombras
    img_cloud_shadow = add_shadow_bands(img_cloud)
    
    # Combinar nubes y sombras (valor 1 = nube/sombra, 0 = despejado)
    is_cld_shdw = (img_cloud_shadow.select('clouds')
                   .add(img_cloud_shadow.select('shadows'))
                   .gt(0))
    
    # Eliminar parches pequeños y dilatar con BUFFER
    is_cld_shdw = (is_cld_shdw
                   .focalMin(2)
                   .focalMax(BUFFER * 2 / 20)
                   .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
                   .rename('cloudmask'))
    
    # Agregar máscara final
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    """Aplica la máscara de nubes/sombras a las bandas de reflectancia."""
    # Invertir máscara (0 = nube/sombra, 1 = despejado)
    not_cld_shdw = img.select('cloudmask').Not()
    
    # Aplicar máscara solo a bandas de reflectancia (B.*)
    return img.select('B.*').updateMask(not_cld_shdw)


print("✓ Funciones de enmascaramiento definidas:")
print("  - add_cloud_bands()")
print("  - add_shadow_bands()")
print("  - add_cld_shdw_mask()")
print("  - apply_cld_shdw_mask()")


In [ ]:
# ============================================================================
# GENERAR MOSAICO COMPUESTO (MEDIANA) LIBRE DE NUBES
# ============================================================================

print("\n🎨 Generando mosaico libre de nubes...")

# Aplicar máscaras a la colección y reducir por mediana
s2_sr_median = (s2_sr_cld_col
                .map(add_cld_shdw_mask)
                .map(apply_cld_shdw_mask)
                .median()
                .clip(AOI))  # Recortar al límite del municipio

print("✓ Mosaico generado exitosamente")
print(f"  Método de composición: Mediana")
print(f"  Bandas disponibles: {s2_sr_median.bandNames().getInfo()}")

# Calcular índices espectrales
print("\n📊 Calculando índices espectrales...")

# NDVI = (NIR - Red) / (NIR + Red)
ndvi = s2_sr_median.normalizedDifference(['B8', 'B4']).rename('NDVI')

# NDWI = (Green - NIR) / (Green + NIR) 
ndwi = s2_sr_median.normalizedDifference(['B3', 'B8']).rename('NDWI')

# EVI = 2.5 * ((NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1))
evi = s2_sr_median.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': s2_sr_median.select('B8'),
        'RED': s2_sr_median.select('B4'),
        'BLUE': s2_sr_median.select('B2')
    }).rename('EVI')

# Agregar índices al mosaico
s2_sr_median = s2_sr_median.addBands([ndvi, ndwi, evi])

print("✓ Índices calculados:")
print("  - NDVI (vegetación)")
print("  - NDWI (agua/humedad)")
print("  - EVI (vegetación mejorado)")


## 7. Visualización Interactiva

Creamos un mapa interactivo con folium para visualizar el mosaico y los índices espectrales.


In [ ]:
# ============================================================================
# CONFIGURAR MÉTODO DE VISUALIZACIÓN CON FOLIUM
# ============================================================================

import folium

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    """Método para agregar capas de Earth Engine a un mapa folium."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
    ).add_to(self)

# Agregar el método a folium.Map
folium.Map.add_ee_layer = add_ee_layer

print("✓ Método de visualización configurado")


In [ ]:
# ============================================================================
# CREAR MAPA INTERACTIVO
# ============================================================================

print("\n🗺️  Creando mapa interactivo...")

# Calcular centro del AOI
center = [centroid.y, centroid.x]

# Crear mapa base
m = folium.Map(location=center, zoom_start=11)

# CAPAS DE VISUALIZACIÓN

# 1. Composición RGB (Color verdadero)
m.add_ee_layer(
    s2_sr_median,
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.2},
    '🌍 RGB (color verdadero)',
    True, 1, 0
)

# 2. Composición Falso Color (NIR-Red-Green)
m.add_ee_layer(
    s2_sr_median,
    {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3500, 'gamma': 1.1},
    '🌲 Falso color (NIR-R-G)',
    False, 1, 0
)

# 3. NDVI (Índice de vegetación)
m.add_ee_layer(
    s2_sr_median.select('NDVI'),
    {'min': -0.2, 'max': 0.9, 'palette': ['brown', 'yellow', 'green', 'darkgreen']},
    '🌿 NDVI',
    False, 0.8, 0
)

# 4. NDWI (Índice de agua)
m.add_ee_layer(
    s2_sr_median.select('NDWI'),
    {'min': -0.3, 'max': 0.5, 'palette': ['red', 'white', 'blue']},
    '💧 NDWI',
    False, 0.8, 0
)

# 5. EVI (Vegetación mejorado)
m.add_ee_layer(
    s2_sr_median.select('EVI'),
    {'min': -0.2, 'max': 1.0, 'palette': ['white', 'lightgreen', 'green', 'darkgreen']},
    '📈 EVI',
    False, 0.8, 0
)

# Agregar control de capas
m.add_child(folium.LayerControl())

# Agregar marcador del centroide
folium.Marker(
    location=center,
    popup=f"<b>{area_nombre}</b><br>AOI",
    tooltip=area_nombre,
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

print("✓ Mapa creado exitosamente")
print(f"\n📍 Área: {area_nombre}")
print(f"   Centro: {centroid.y:.4f}, {centroid.x:.4f}")
print("\n💡 Usa el control de capas (esquina superior derecha) para alternar visualizaciones")

# Mostrar mapa
display(m)


## 8. Estadísticas y Descarga de Resultados

Calcula estadísticas de los índices espectrales y permite descargar los resultados.


In [ ]:
# ============================================================================
# CALCULAR ESTADÍSTICAS DEL MOSAICO
# ============================================================================

print("\n📊 Calculando estadísticas del mosaico...\n")

# Calcular estadísticas para cada índice
indices = ['NDVI', 'NDWI', 'EVI']
stats = {}

for idx_name in indices:
    print(f"Calculando {idx_name}...")
    idx_stats = s2_sr_median.select(idx_name).reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ).combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        ),
        geometry=AOI,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    
    stats[idx_name] = {
        'mean': idx_stats.get(f'{idx_name}_mean'),
        'stdDev': idx_stats.get(f'{idx_name}_stdDev'),
        'min': idx_stats.get(f'{idx_name}_min'),
        'max': idx_stats.get(f'{idx_name}_max')
    }

print("\n" + "="*70)
print(f"ESTADÍSTICAS - {area_nombre} ({START_DATE} a {END_DATE})")
print("="*70)

for idx_name, values in stats.items():
    if values['mean'] is not None:
        print(f"\n{idx_name}:")
        print(f"  Media:   {values['mean']:.4f}")
        print(f"  Std Dev: {values['stdDev']:.4f}")
        print(f"  Mínimo:  {values['min']:.4f}")
        print(f"  Máximo:  {values['max']:.4f}")
    else:
        print(f"\n{idx_name}: No hay datos disponibles")

print("\n" + "="*70)

# Crear DataFrame con resultados
import pandas as pd

results_df = pd.DataFrame({
    'Area': [area_nombre],
    'Fecha_inicio': [START_DATE],
    'Fecha_fin': [END_DATE],
    'N_imagenes': [n_images],
    'Area_ha': [total_area_ha],
    'NDVI_mean': [stats['NDVI']['mean']],
    'NDVI_std': [stats['NDVI']['stdDev']],
    'NDWI_mean': [stats['NDWI']['mean']],
    'NDWI_std': [stats['NDWI']['stdDev']],
    'EVI_mean': [stats['EVI']['mean']],
    'EVI_std': [stats['EVI']['stdDev']]
})

print("\n✓ Estadísticas calculadas")


In [ ]:
# ============================================================================
# DESCARGAR ESTADÍSTICAS (CSV)
# ============================================================================

print("\n💾 Preparando descarga de estadísticas...")

# Guardar CSV
csv_filename = f'estadisticas_{area_nombre.replace(" ", "_")}_{START_DATE}_{END_DATE}.csv'
results_df.to_csv(csv_filename, index=False)

print(f"✓ Archivo CSV creado: {csv_filename}")

# Descargar en Colab
if IN_COLAB:
    from google.colab import files
    files.download(csv_filename)
    print("✓ Descarga iniciada en tu navegador")
else:
    print(f"✓ Archivo guardado localmente: {csv_filename}")

# Mostrar tabla
print("\n📋 Resumen de resultados:")
print(results_df.to_string(index=False))


### Exportar imagen a Google Drive (Opcional)

Si deseas exportar el mosaico completo como GeoTIFF, ejecuta la siguiente celda. La imagen se guardará en tu Google Drive.


In [ ]:
# ============================================================================
# EXPORTAR MOSAICO A GOOGLE DRIVE (OPCIONAL)
# ============================================================================

# Seleccionar bandas a exportar
bands_to_export = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI', 'NDWI', 'EVI']
image_to_export = s2_sr_median.select(bands_to_export)

# Configurar exportación
export_name = f'S2_Mosaic_{area_nombre.replace(" ", "_")}_{START_DATE}_{END_DATE}'

task = ee.batch.Export.image.toDrive(
    image=image_to_export,
    description=export_name,
    folder='EarthEngine_Exports',
    fileNamePrefix=export_name,
    scale=10,  # Resolución en metros
    region=AOI,
    maxPixels=1e9,
    crs='EPSG:4326',
    fileFormat='GeoTIFF'
)

# Iniciar tarea
task.start()

print("="*70)
print("EXPORTACIÓN INICIADA")
print("="*70)
print(f"  Nombre: {export_name}")
print(f"  Carpeta: EarthEngine_Exports (en tu Google Drive)")
print(f"  Bandas: {', '.join(bands_to_export)}")
print(f"  Resolución: 10 m")
print(f"  Área: {total_area_ha:.2f} ha")
print("="*70)
print("\n⏳ La exportación se está procesando en segundo plano.")
print("   Revisa el progreso en: https://code.earthengine.google.com/tasks")
print("   El archivo aparecerá en Google Drive cuando termine.")
